In [1]:
# from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings.huggingface import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import Chroma

class EmbeddingManager:
    def __init__(self, all_sections, persist_directory='./.markdown'):
        self.all_sections = all_sections
        self.persist_directory = persist_directory
        self.vectordb = None
        
    # Method to create and persist embeddings
    def create_and_persist_embeddings(self):
        # Creating an instance of OpenAIEmbeddings
        # embedding = OpenAIEmbeddings()

        embedding = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en-v1.5")
        def split_list(input_list, chunk_size):
            for i in range(0, len(input_list), chunk_size):
                yield input_list[i:i + chunk_size]

        split_docs_chunked = split_list(self.all_sections, 5000)

        # Creating an instance of Chroma with the sections and the embeddings
        for split_docs_chunk in split_docs_chunked:
            self.vectordb = Chroma.from_documents(
                documents=split_docs_chunk,
                embedding=embedding,
                persist_directory=self.persist_directory,
            )
            # Persisting the embeddings
            self.vectordb.persist()

In [3]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import utils as chromautils

class DocumentManager:
    def __init__(self, file_path):
        self.file_path = file_path
        self.documents = []
        self.all_sections = []
    
    def load_documents(self):
        # loader = DirectoryLoader(self.directory_path, glob=self.glob_pattern, show_progress=True, loader_cls=UnstructuredMarkdownLoader)
        loader = UnstructuredMarkdownLoader(self.file_path, mode="elements")
        documents = loader.load()
        self.documents = chromautils.filter_complex_metadata(documents)
        # text_splitter = RecursiveCharacterTextSplitter(
        #             chunk_size=10000, chunk_overlap=200, add_start_index=True
        #     )
        # self.documents = text_splitter.split_documents(filter_documents)

    def split_documents(self):
        for doc in self.documents:
            self.all_sections.append(doc)

In [4]:
# Initialising and loading documents
doc_manager = DocumentManager('Z:/github/qachatbot/qachatbot/.markdown/5esrd.md')
doc_manager.load_documents()
doc_manager.split_documents()

In [5]:
len(doc_manager.all_sections)

12933

In [7]:
doc_manager.all_sections[-100]

Document(metadata={'source': 'Z:/github/qachatbot/qachatbot/.markdown/5esrd.md', 'last_modified': '2024-07-18T21:16:39', 'parent_id': '3b75181130dc6c40c6652862bc9df07f', 'filetype': 'text/markdown', 'file_directory': 'Z:/github/qachatbot/qachatbot/.markdown', 'filename': '5esrd.md', 'category': 'NarrativeText'}, page_content='Rapier. Melee Weapon Attack: +3 to hit, reach 5 ft., one target. Hit: 5 (1d8 + 1) piercing damage.')

In [8]:
# Creation and persistence of embeddings
embed_manager = EmbeddingManager(doc_manager.all_sections)
embed_manager.create_and_persist_embeddings()

z:\venv\qachatbot\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
z:\venv\qachatbot\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [9]:
# retrieve
retriever = embed_manager.vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 2})
retrieved_docs = retriever.invoke("Barbarian hit dice?")
retrieved_docs

[Document(metadata={'category': 'Title', 'category_depth': 0, 'file_directory': 'Z:/github/qachatbot/qachatbot/.markdown', 'filename': '5esrd.md', 'filetype': 'text/markdown', 'last_modified': '2024-07-18T21:16:39', 'source': 'Z:/github/qachatbot/qachatbot/.markdown/5esrd.md'}, page_content='Hit Dice: 1d12 per barbarian level'),
 Document(metadata={'category': 'NarrativeText', 'file_directory': 'Z:/github/qachatbot/qachatbot/.markdown', 'filename': '5esrd.md', 'filetype': 'text/markdown', 'last_modified': '2024-07-18T21:16:39', 'parent_id': '53a68f3d59e096bf768a98b9d4e0fd62', 'source': 'Z:/github/qachatbot/qachatbot/.markdown/5esrd.md'}, page_content='Hit Points at Higher Levels: 1d12 (or 7) + your Constitution\nmodifier per barbarian level after 1st')]

In [10]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            (
                "You are an assistant for question-answering tasks. "
                "Use the following pieces of retrieved context to answer the question. "
                "If you don't know the answer, just say that you don't know. "
                "Use three sentences maximum and keep the answer concise. \n"
                "Context: {context} \n"
                "Question: {question} \n"
                "Answer:"
            ),
        ),
    ]
)

In [11]:
from langchain_community.chat_models.ollama import ChatOllama
from langchain_core.language_models.chat_models import BaseChatModel

llm = ChatOllama(model="phi3")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def invoke_chat(user_input: str, k=3, llm: BaseChatModel = None):
    context = retriever.invoke(user_input, k=k)
    context = format_docs(context)
    message = prompt.invoke({
        "context": context,
        "question": user_input
    })
    if llm:
        message = llm.invoke(message)
    return message

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt
)

In [13]:
rag_chain.invoke("What is Barbarian hit point?")

ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. \nContext: Hit Points at Higher Levels: 1d12 (or 7) + your Constitution\nmodifier per barbarian level after 1st\n\nBarbarian \nQuestion: What is Barbarian hit point? \nAnswer:")])

In [16]:
response = invoke_chat("What is Barbarian hit point?", k=3, llm=llm)

In [17]:
print(response.content)

 The formula for a Barbarian's Hit Points at higher levels is either "1d12 (roll to get number) + your Constitution modifier" per barbarian level after the first. If not specified, they start with 10 HD points plus their Constitution modifier as base hit point count before any roll or additional bonuses are applied.
